![logo from Mint.com](images/Mint_Lockup_1_.png)

# Mint Trends

## Glossary
**Net Worth** - renamed from "Net" on Mint; Assets minus Debts

**Net Income** - Income minus Spending

**Savings Rate** - (Income minus Spending) divided by Income

**Growth** - difference in Asset value from the previous month

**Unearned Income** - Growth that is not accounted for by Net Income.

**Total Income** - Income plus Unearned Income; alternatively, Growth plus Spending (Note: Spending is represented as a postiive value in Mint)

In [ ]:
import re
import locale
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta


locale.setlocale(locale.LC_ALL, '')


def currency_to_float(arg):
    value = re.sub(r'[$,]', '', arg)
    return float(value)

def date_string_to_datetime(arg):
    return datetime.strptime(arg, '%B %Y')

def float_to_currency(arg):
    return locale.currency(arg)

converters = {
    'Dates': date_string_to_datetime,
    'Assets': currency_to_float,
    'Debts': currency_to_float,
    'Net': currency_to_float,
    'Income': currency_to_float,
    'Spending': currency_to_float,
}

ONE_YEAR_AGO = (datetime.today() - timedelta(days=365)).date().isoformat()
filter_date = ONE_YEAR_AGO[:-3] # Or replace with custom 'YYYY-MM' value

In [ ]:
net_worth = pd.read_csv('data/net_worth.csv', converters=converters, index_col='Dates') \
    .rename(columns={'Net': 'Net Worth'})
income = pd.read_csv('data/income.csv', converters=converters, engine='python', skipfooter=1)
spending = pd.read_csv('data/spending.csv', converters=converters, engine='python', skipfooter=1)

income.index = income['Dates']
income['Month'] = income['Dates'].apply(lambda x: x.strftime('%b %y'))
income = income.drop(columns=['Dates'])
spending.index = spending['Dates']
spending = spending.drop(columns=['Dates'])

combined = pd.concat([income, spending, net_worth], axis='columns')
combined['Net Income'] = combined['Income'] - combined['Spending']
combined['Savings Rate'] = np.round((combined['Income'] - combined['Spending']) / combined['Income'] * 100, 2)
combined['Growth'] = combined['Assets'].diff()
combined['Unearned Income'] = combined['Growth'] - combined['Net Income']
combined['Total Income'] = combined['Income'] + combined['Unearned Income']  # Or Growth + Spending

# Filter after specified date
combined = combined[combined.index >= filter_date]
combined.tail(20)

In [ ]:
assets_now = combined.iloc[len(combined) - 1]['Assets']
assets_then = combined.iloc[0]['Assets']
asset_growth = np.round(assets_now / assets_then * 100, 2)

# Look back one extra month for current income
total_income_now = np.round(combined.iloc[len(combined) - 2]['Total Income'], 2)
total_income_then = np.round(combined.iloc[0]['Total Income'], 2)
income_growth = np.round(total_income_now / total_income_then * 100, 2)

report = combined.drop(columns=['Debts','Assets', 'Month', 'Net Worth', 'Savings Rate'])

print(f"Differences since {filter_date}:\n"
      f"Your Assets have grown {asset_growth}% ("
      f"{float_to_currency(total_income_then)} => {float_to_currency(total_income_now)}).\n"
      "Your Savings Rate in this time was "
      f"{np.round((report['Income'].sum() - report['Spending'].sum()) / report['Income'].sum() * 100, 2)}%.")
print()
print(report.sum().apply(float_to_currency))
print()

print(f"Monthly averages since {filter_date}:\n"
      f"{np.round(report['Unearned Income'].sum() / report['Growth'].sum() * 100, 2)}% of your Growth and " 
      f"{np.round(report['Unearned Income'].sum() / report['Total Income'].sum() * 100, 2)}% " 
      "of your Total Income is from Unearned Income.\nUnearned Income covers " 
      f"{np.round(report['Unearned Income'].sum() / report['Spending'].sum() * 100, 2)}% "
      "of your Spending.")
print()
print(report.mean().apply(float_to_currency))
print()


In [ ]:
# Create long data set to utilize Seaborn's categorical plotting features
long_combined = combined[['Income', 'Unearned Income']] \
    .melt(ignore_index=False)

plt.figure(figsize = (16,9))
g = sns.barplot(x=long_combined.index, y="value",
             hue="variable",
             data=long_combined)
g.set_xticklabels(combined["Month"])
plt.title(f"Income over Time since {filter_date}", fontsize = 20)
plt.xlabel("Month", fontsize = 15)
plt.ylabel("Total", fontsize = 15)
plt.show()


In [ ]:
# Create long data set to utilize Seaborn's categorical plotting features
long_combined = combined[['Income', 'Unearned Income', 'Spending']] \
    .melt(ignore_index=False)

plt.figure(figsize = (16,9))
g = sns.lineplot(x=long_combined.index, y="value",
             hue="variable",
             data=long_combined)
plt.title(f"Income and Spending over Time since {filter_date}", fontsize = 20)
plt.xlabel("Month", fontsize = 15)
plt.ylabel("Total", fontsize = 15)
plt.show()


In [ ]:
plt.figure(figsize = (16,9))
g = sns.lineplot(x=combined.index, y="Savings Rate", data=combined)
plt.title(f"Savings Rate over Time since {filter_date}", fontsize = 20)
plt.xlabel("Month", fontsize = 15)
plt.ylabel("Total", fontsize = 15)
plt.show()

In [ ]:
plt.figure(figsize = (16,4))
g = sns.barplot(x=combined.index, y="Total Income", data=combined)
g.set_xticklabels(combined["Month"])
plt.title("Total Income over Time", fontsize = 20)
plt.xlabel("Month", fontsize = 15)
plt.ylabel("Total", fontsize = 15)
plt.show()

In [ ]:
plt.figure(figsize = (16,4))
g = sns.lineplot(x=combined.index, y="Net Worth", data=combined)
plt.title("Net Worth over Time", fontsize = 20)
plt.xlabel("Month", fontsize = 15)
plt.ylabel("Total", fontsize = 15)
plt.show()

In [ ]:
plt.figure(figsize = (16,4))
g = sns.barplot(x=combined.index, y="Growth", data=combined)
g.set_xticklabels(combined["Month"])
plt.title("Growth over Time", fontsize = 20)
plt.xlabel("Month", fontsize = 15)
plt.ylabel("Total", fontsize = 15)
plt.show()

In [ ]:
plt.figure(figsize = (16,4))
g = sns.barplot(x=combined.index, y="Unearned Income", data=combined)
g.set_xticklabels(combined["Month"])
plt.title("Unearned Income over Time", fontsize = 20)
plt.xlabel("Month", fontsize = 15)
plt.ylabel("Total", fontsize = 15)
plt.show()